# Motivational Example

This notebook is used to run and analyze a toy example of `WA-v4` as the motivation of proposing MSTCs. More details can be referred to Section 4 of [our TOSEM paper](https://dl.acm.org/doi/abs/10.1145/3736757).

**Tips**： 
+ We should ensure that the current working directory is `.\mstcs`, due the inclusion of `os.getcwd()` to obtain the absolute path in Jupyter Notebook.
+ This notebook is not scalable (for `WA-v4`), and only used for replication and reproduction.

In [1]:
# Import qiskit-related packages
from qiskit import QuantumCircuit 
 
# Import basic packages
import math
import numpy as np

# Import packages within our repository
import os, sys
current_dir = os.getcwd()
parent_dir = os.path.dirname(current_dir)
sys.path.append(parent_dir)   
from mycode.utils import generate_numbers, outputdict2samps, outputdict2probs, OPO_UTest, circuit_execution

The following code can examine the test performance of pure-state test cases (PSTCs). Upon covering all the computation basis states (a.k.a. classical states in some papers), only two PSTCs trigger the failures.

In [2]:
from mycode.testing.WeightedAdder.utils.adder_specification import PSTC_specification
from mycode.testing.WeightedAdder.programs import WeightedAdder_defect4, WeightedAdder
from mycode.testing.WeightedAdder.config import default_shots, candidate_initial_states
import csv  

# ===================================================
# Configuration parameters
# ===================================================
n = 3
weight = [1, 1, 0]

initial_states_list = generate_numbers(n, len(candidate_initial_states))
pre_time = 0  # Time tracking for state preparation phase

# Calculate required number of output qubits (s)
if np.sum(weight) == 0:
    s = 1
else:
    s = 1 + math.floor(math.log2(np.sum(weight)))

# Initialize quantum circuit instances
qc_test = WeightedAdder_defect4(n, weight)  # Defective implementation
qc_raw = WeightedAdder(n, weight)           # Reference implementation

# Switch to ./data/motivational_examples
output_path = os.path.join(parent_dir, "data", "motivational_examples")
os.makedirs(output_path, exist_ok=True)
os.chdir(output_path)

# Create CSV file and write header
csv_filename = "PSTCs_results_for_motivation.csv"
with open(csv_filename, 'w', newline='') as csvfile:
    csv_writer = csv.writer(csvfile)
    # Write column headers
    csv_writer.writerow(['quantum_input_ID', 'test_probs', 'exp_probs', 'test_results'])
    
    # Iterate through all possible initial states
    for initial_states in initial_states_list:
        # Convert binary state to decimal number
        number = int(''.join(map(str, initial_states)), 2)
        initial_states = initial_states[::-1]  # Reverse for Qiskit's qubit ordering
        
        # Initialize quantum circuit
        qc = QuantumCircuit(qc_test.num_qubits, s)
        
        # Apply X gates for state preparation
        for index, val in enumerate(initial_states):
            if candidate_initial_states[val] == 1:
                qc.x(index)
        
        # Append the test circuit and set measurement
        qc.append(qc_test, qc.qubits) # type: ignore
        qc.measure(qc.qubits[n: n + s], qc.clbits)

        # Execute circuit on simulator
        dict_counts = circuit_execution(qc, default_shots)
        test_probs = outputdict2probs(dict_counts, s)  # Calculate probability distribution
        test_samps = outputdict2samps(dict_counts)     # Transfer the test samples

        # Generate expected probability distribution
        exp_probs = PSTC_specification(s, initial_states, weight)
        exp_samps = list(np.random.choice(range(2 ** qc.num_clbits), 
                         size=default_shots, p=exp_probs))

        # Perform statistical hypothesis test
        test_result = OPO_UTest(exp_samps, test_samps)
        
        # Format probabilities for CSV storage
        test_probs_str = ';'.join([f"{p:.6f}" for p in test_probs])  # Semicolon-separated
        exp_probs_str = ';'.join([f"{p:.6f}" for p in exp_probs])   # to prevent CSV formatting issues
        
        # Write results to CSV
        csv_writer.writerow([number, test_probs_str, exp_probs_str, test_result])
        
        # Maintain console output for real-time monitoring
        print(
            f"quantum_input_ID={number}, "
            f"test_probs={test_probs}, "
            f"exp_probs={exp_probs}, "
            f"test_result={test_result}"
        )

quantum_input_ID=0, test_probs=[1. 0. 0. 0.], exp_probs=[1, 0.0, 0.0, 0.0], test_result=pass
quantum_input_ID=1, test_probs=[0. 1. 0. 0.], exp_probs=[0.0, 1, 0.0, 0.0], test_result=pass
quantum_input_ID=2, test_probs=[0. 1. 0. 0.], exp_probs=[0.0, 1, 0.0, 0.0], test_result=pass
quantum_input_ID=3, test_probs=[0. 0. 0. 1.], exp_probs=[0.0, 0.0, 1, 0.0], test_result=fail
quantum_input_ID=4, test_probs=[1. 0. 0. 0.], exp_probs=[1, 0.0, 0.0, 0.0], test_result=pass
quantum_input_ID=5, test_probs=[0. 1. 0. 0.], exp_probs=[0.0, 1, 0.0, 0.0], test_result=pass
quantum_input_ID=6, test_probs=[0. 1. 0. 0.], exp_probs=[0.0, 1, 0.0, 0.0], test_result=pass
quantum_input_ID=7, test_probs=[0. 0. 0. 1.], exp_probs=[0.0, 0.0, 1, 0.0], test_result=fail


Then, draw the involved quantum circuit diagrams to the path `./data/motivational_examples`. The files `qc_raw.pdf` and `decom1_qc_test.pdf` are the two illustrated in Fig.2. 

In [3]:
# Draw the quantum circuit diagram
import matplotlib.pyplot as plt
import  os

# Switch to ./data/motivational_examples
output_path = os.path.join(parent_dir, "data", "motivational_examples")
os.makedirs(output_path, exist_ok=True)
os.chdir(output_path)

def circuit_drawing(qc: QuantumCircuit, file_name: str) -> None:
    '''  
    The argument 'file_name' determines the name the file to be saved.
    '''
    fig = qc.draw(output='mpl', style='clifford')  
    fig.set_size_inches(10, 6)              # type: ignore # Set the figure size
    plt.rcParams.update({'font.size': 12})  # Set the front size
    # Save as .pdf
    fig.savefig('{}.pdf'.format(file_name), bbox_inches='tight', dpi=600) # type: ignore
    return

# Print quantum circuits
decom1_qc_raw = qc_raw.decompose()
decom1_qc_test = qc_test.decompose()
circuit_drawing(qc_raw, "qc_raw")                  # Quantum circuit of the bug-free version
circuit_drawing(decom1_qc_raw, "decom1_qc_raw")    # Quantum circuit of the bug-free version with one decomposition
circuit_drawing(decom1_qc_test, "decom1_qc_test")  # Quantum circuit of the buggy version with one decomposition